## Tasa de respuestas correctas LLMs

In [1]:
#!pip install streamlit
# #https://dashboard.ngrok.com/signup
#!pip install --upgrade typing_extensions
#!pip install openai
#!pip install pypdf
#!pip install langchain

In [2]:
#pip install -U langchain-community

In [3]:
# Importe la clase PyPDFLoader del módulo langchain.document_loaders.
from langchain.document_loaders import PyPDFLoader

# Cree una instancia de la clase Py
# PDFLoader, pasando el nombre del archivo del documento PDF que desea cargar.
loader = PyPDFLoader("llm_doc.pdf")

# Este método lee el contenido del archivo PDF especificado al crear la instancia del cargador.
# El contenido del documento cargado se almacena en la variable 'documentos' para su posterior procesamiento.
documents = loader.load()

In [4]:
# Esta clase se utiliza para dividir textos en fragmentos más pequeños, basándose en el número de caracteres.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Se especifica el tamaño de cada fragmento de texto (chunk_size) en 1000 caracteres y
# la superposición entre fragmentos consecutivos (chunk_overlap) en 100 caracteres.
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 1000,
                                               chunk_overlap = 100)

# Este método divide el contenido de los documentos cargados en fragmentos más pequeños,
# basándose en los parámetros de tamaño y superposición especificados.
doc_splits = text_splitter.split_documents(documents)

In [5]:
#pip install --upgrade pandas

In [6]:
#pip install numpy

In [7]:
doc_splits[0].metadata

{'producer': 'Adobe PDF Library 17.0',
 'creator': 'Adobe InDesign 19.0 (Windows)',
 'creationdate': '2023-11-27T11:41:44+05:30',
 'moddate': '2023-11-27T11:41:45+05:30',
 'trapped': '/False',
 'source': 'llm_doc.pdf',
 'total_pages': 14,
 'page': 0,
 'page_label': '1'}

In [8]:
import pandas as pd
# Crear una lista de diccionarios a partir de los fragmentos de documento divididos previamente.
# Cada diccionario contiene dos claves: 'Chunks', que almacena el contenido del fragmento de texto,
# y 'Metadata', que almacena los metadatos asociados a dicho fragmento.
data = [{'Chunks': doc.page_content, 'Metadata': doc.metadata} for doc in doc_splits]
# Crear un DataFrame de pandas a partir de la lista de diccionarios 'data'.
df_vector_store = pd.DataFrame(data)
df_vector_store.head()

,Chunks,Metadata
0,Building Pipelines and Environments for \nLar...,"{'producer': 'Adobe PDF Library 17.0', 'creato..."
1,Contents\nIntroduction to LLMOps 1\nWhy LLMOps...,"{'producer': 'Adobe PDF Library 17.0', 'creato..."
2,1.\nIntroduction to LLMOps\nGenerative AI mode...,"{'producer': 'Adobe PDF Library 17.0', 'creato..."
3,translation and even creative writing.\nUsing ...,"{'producer': 'Adobe PDF Library 17.0', 'creato..."
4,"with LLMs. Together, these allow data scientis...","{'producer': 'Adobe PDF Library 17.0', 'creato..."


In [9]:
#pip install openai==0.28

In [10]:
from openai import AzureOpenAI
import pandas as pd
import numpy as np
import json

# === 1. Cargar credenciales desde JSON ===
def cargar_credenciales(ruta_credenciales):
    with open(ruta_credenciales, "r") as file:
        return json.load(file)

# === 2. Crear cliente AzureOpenAI ===
def crear_cliente_azure(creds):
    client = AzureOpenAI(
        api_key=creds["AZURE_API_KEY"],
        api_version=creds["AZURE_API_VERSION"],
        azure_endpoint=creds["AZURE_ENDPOINT"]
    )
    return client

# === 3. Generar embeddings usando cliente (AzureOpenAI) ===
def text_embedding(text=[]):
    embeddings = client.embeddings.create(model="text-embedding-ada-002",
                                          input=text,
                                          encoding_format="float")
    return embeddings.data[0].embedding

# === 4. Uso real del código ===
ruta_credenciales = "credentials.json"
creds = cargar_credenciales(ruta_credenciales)
client = crear_cliente_azure(creds)

# Aplicar embeddings
df_vector_store["Embedding"] = df_vector_store["Chunks"].apply(lambda x: text_embedding([x]))
df_vector_store["Embedding"] = df_vector_store["Embedding"].apply(np.array)

# Guardar a archivo
#df_vector_store.to_pickle('df_vector_store.pkl')

In [11]:
df_vector_store.head()


,Chunks,Metadata,Embedding
0,Building Pipelines and Environments for \nLar...,"{'producer': 'Adobe PDF Library 17.0', 'creato...","[-0.001881948, -0.012164683, 0.016830197, -0.0..."
1,Contents\nIntroduction to LLMOps 1\nWhy LLMOps...,"{'producer': 'Adobe PDF Library 17.0', 'creato...","[0.0030545578, -0.0043646125, -0.0022074673, -..."
2,1.\nIntroduction to LLMOps\nGenerative AI mode...,"{'producer': 'Adobe PDF Library 17.0', 'creato...","[-0.021225467, -0.008492879, -0.013970853, -0...."
3,translation and even creative writing.\nUsing ...,"{'producer': 'Adobe PDF Library 17.0', 'creato...","[-0.0065719597, -0.0055598076, -0.0033509966, ..."
4,"with LLMs. Together, these allow data scientis...","{'producer': 'Adobe PDF Library 17.0', 'creato...","[0.0069066356, -0.009416879, 0.0025986563, -0...."


In [12]:
# Definir una cadena de texto que contiene la consulta de interés.
#query = '¿Cómo se selecciona un modelo llm?'

query = '¿Qué son los LLMOps?'

# Generar la representación vectorial (embedding) de la consulta de texto utilizando la función 'text_embedding'.
query_embedding = text_embedding(query)

In [13]:
# Definir una función para calcular el producto punto entre dos vectores.
def get_dot_product(row):
    return np.dot(row, query_vector)

# Definir una función para calcular la similitud de coseno entre dos vectores.
def cosine_similarity(row):
    denominator1 = np.linalg.norm(row)
    denominator2 = np.linalg.norm(query_vector.ravel())
    dot_prod = np.dot(row, query_vector)
    return dot_prod/(denominator1*denominator2)

# Definir una función para obtener los fragmentos de texto más relevantes desde un almacenamiento
# vectorial dada una consulta.
def get_context_from_query(query, vector_store, n_chunks = 5):
    global query_vector
    # Convertir el embedding de la consulta en un array de numpy.
    query_vector = np.array(query_embedding)
    # Calcular la similitud de coseno para cada vector en el almacenamiento y ordenar los resultados
    # de mayor a menor similitud, seleccionando los índices de los 'n_chunks' más altos.
    top_matched = (
        vector_store["Embedding"]
        .apply(cosine_similarity)
        .sort_values(ascending=False)[:n_chunks]
        .index)
    # Seleccionar los fragmentos de texto correspondientes a los índices de mayor similitud.
    top_matched_df = vector_store[vector_store.index.isin(top_matched)][["Chunks"]]
    # Devolver una lista con los fragmentos de texto seleccionados.
    return list(top_matched_df['Chunks'])

# Utilizar la función 'get_context_from_query' para obtener los fragmentos de texto más relevantes
# dada la consulta especificada y el almacenamiento vectorial 'df_vector_store', limitando el número
# de fragmentos a 5.
Context_List = get_context_from_query(
    query        = query,
    vector_store = df_vector_store,
    n_chunks     = 5)


In [14]:
custom_prompt = """
Eres una Inteligencia Artificial super avanzada que trabaja asistente personal.
Utilice los RESULTADOS DE BÚSQUEDA SEMANTICA para responder las preguntas del usuario.
Solo debes utilizar la informacion de la BUSQUEDA SEMANTICA si es que hace sentido y tiene relacion con la pregunta del usuario.
Si la respuesta no se encuentra dentro del contexto de la búsqueda semántica, no inventes una respuesta, y responde amablemente que no tienes información para responder.

RESULTADOS DE BÚSQUEDA SEMANTICA:
{source}

Lee cuidadosamente las instrucciones, respira profundo y escribe una respuesta para el usuario!
""".format(source = str(Context_List))

In [15]:
# Models
# modelo_banco = "gpt-4o-mini_clasificacion-PQRS"
# modelo_banco = "gpt-35-turbo-16k-PQR"

modelo_banco = "gpt-35-turbo-16k-PQR"

completion = client.chat.completions.create(
  model=modelo_banco,
  temperature = 0.0,
  messages=[
    {"role": "system", "content": custom_prompt},
    {"role": "user", "content": query}
  ]
)

# Imprimir el contenido del mensaje de la primera opción de completación generada por el modelo.
print(completion.choices[0].message.content)

Los LLMOps, o Large Language Model Operations, son un marco de herramientas y mejores prácticas diseñadas para gestionar el ciclo de vida de las aplicaciones impulsadas por modelos de lenguaje de gran tamaño, desde el desarrollo hasta la implementación y el mantenimiento. Estas herramientas se centran en la experimentación, implementación, gestión y monitoreo de grandes modelos de lenguaje, con el objetivo de permitir capacidades de inteligencia artificial con estos modelos, desarrollando mejores indicaciones, contextos más extensos, inferencias más rápidas y técnicas personalizadas que permitan una experimentación e innovación rápidas con los modelos de lenguaje de gran tamaño.


## Tasa de respuestas correctas

In [16]:
import pandas as pd
base = pd.read_excel("preguntas_BOT.xlsx")
base.head(2)

,Item,Integrante,Pregunta,Respuesta
0,P1,Katherine,¿Qué son los LLMOps?,Los LLMOps son un marco de herramientas y mejo...
1,P2,Katherine,¿Por qué son importantes los LLMOps?,Los LLMOps son esenciales para superar desafío...


In [17]:
es_correcta=[]
respuestas = []

for i in range(10):
    query = base["Pregunta"][i]
    respuesta_humana = base["Respuesta"][i]

    query_embedding = text_embedding(query)
    query_vector = np.array(query_embedding)

    def cosine_similarity(row_embedding):
        denominator1 = np.linalg.norm(row_embedding)
        denominator2 = np.linalg.norm(query_vector)
        dot_prod = np.dot(row_embedding, query_vector)
        return dot_prod/(denominator1*denominator2)

    top_matched=(
        df_vector_store["Embedding"].apply(cosine_similarity).sort_values(ascending=False)[:5].index
    )

    context_chunks=list(df_vector_store.loc[top_matched,'Chunks'])

    custom_prompt = """
    Eres una Inteligencia Artificial super avanzada que trabaja asistente personal.
    Utilice los RESULTADOS DE BÚSQUEDA SEMANTICA para responder las preguntas del usuario.
    Solo debes utilizar la informacion de la BUSQUEDA SEMANTICA si es que hace sentido y tiene relacion con la pregunta del usuario.
    Si la respuesta no se encuentra dentro del contexto de la búsqueda semántica, no inventes una respuesta, y responde amablemente que no tienes información para responder.

    RESULTADOS DE BÚSQUEDA SEMANTICA:
    {source}

    Lee cuidadosamente las instrucciones, respira profundo y escribe una respuesta para el usuario!
    """.format(source = str(context_chunks))

    completion = client.chat.completions.create(
    model=modelo_banco,
    temperature = 0.0,
    messages=[
        {"role": "system", "content": custom_prompt},
        {"role": "user", "content": query}
    ]
    )

    # Imprimir el contenido del mensaje de la primera opción de completación generada por el modelo.
    respuesta_modelo=completion.choices[0].message.content
    respuestas.append(respuesta_modelo)

    emb_modelo=np.array(text_embedding(respuesta_modelo))
    emb_humana=np.array(text_embedding(respuesta_humana))
    def cosine_(v1,v2):
        return np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    similitud=cosine_(emb_modelo,emb_humana)

    correcta = 1 if similitud>=0.90 else 0
    print("Pregunta"+str(i+1)," Similaridad: ",similitud," Feedback: ",correcta)  
    es_correcta.append(correcta)

Pregunta1  Similaridad:  0.9116775409886054  Feedback:  1
Pregunta2  Similaridad:  0.9463164461463259  Feedback:  1
Pregunta3  Similaridad:  0.9194636226766232  Feedback:  1
Pregunta4  Similaridad:  0.8998629977361134  Feedback:  0
Pregunta5  Similaridad:  0.9476553843912816  Feedback:  1
Pregunta6  Similaridad:  0.9759774868664376  Feedback:  1
Pregunta7  Similaridad:  0.9029540936275927  Feedback:  1
Pregunta8  Similaridad:  0.8601067311907453  Feedback:  0
Pregunta9  Similaridad:  0.8450317926436656  Feedback:  0
Pregunta10  Similaridad:  0.9198931560805396  Feedback:  1


In [18]:
print("La tasa de respuestas correctas es:",100*np.mean(es_correcta))

La tasa de respuestas correctas es: 70.0
